hw9 - rohan bhatt - msml604

In [5]:
import numpy as np
import pandas as pd
from numpy.linalg import norm, solve

#problem data (f(x) = 0.5 * x^T A x + b^T x)
A = np.array([[10.25, 9.75],
              [9.75, 10.25]])
b = np.array([2.0, -4.0])

#objective function
def f(x):
    return 0.5 * x @ A @ x + b @ x

#gradient
def grad_f(x):
    return A @ x + b

#x* and f*, ground truth sol
x_star = -solve(A, b) # x* = -A^-1
f_star = f(x_star) # f(x*)

#optimization loop settings
x0 = np.zeros(2) #starting point (0,0)
max_iterations = 5000 #hard iteration cap
tol = 1e-10 #stop when ||f|| < tol
results = [] #list of dicts, will turn into DF later

#helper to log each optimiser's outcome
def store(name, iters, x_final):
    results.append({
        "Method": name,
        "Iter": iters,  
        "x0": x_final[0], #first coordinate
        "x1": x_final[1], #second coordinate
        "‖x - x*‖": norm(x_final - x_star), #distance to optimum
        "f(x_final)": f(x_final) #objective val reached
    })

# steepest descent with fixed step size
alpha = 0.02
x = x0.copy()
for k in range(1, max_iterations + 1):
    g = grad_f(x) #current gradient
    if norm(g) < tol:  #convergnce test
        break
    x -= alpha * g #gradient step
store("GD (a=0.02)", k, x)

# gradient with momentum (heavy ball)
rho = 0.5 #momentum
eta = .02 #learning rate n
x = x0.copy()
v = np.zeros_like(x) #momentum buffer
for k in range(1, max_iterations + 1):
    g = grad_f(x)
    if norm(g) < tol:
        break
    v = rho * v + g #accumulate gradient into momentum
    x -= eta * v #update with momentum term
store("Momentum (n=0.02, p=0.5)", k, x)

# Nesterov accelerated gradient (look ahead)
eta, rho = 0.02, 0.5
x = x0.copy()
v = np.zeros_like(x)
for k in range(1, max_iterations + 1):
    g = grad_f(x - rho * v)
    if norm(g) < tol:
        break
    v = rho * v + g #same momentum update as before
    x -= eta * v
store("Nesterov (n=0.02, p=0.5)", k, x)

# RMSprop 
eta = .15 #learning rate
eps = .5 #stabiliser e
rho_rms = .8 #decay rate
x = x0.copy()
s = np.zeros_like(x) #running avg of squared grads
for k in range(1, max_iterations + 1):
    g = grad_f(x)
    if norm(g) < tol:
        break
    s = rho_rms * s + (1 - rho_rms) * g**2 #update moving 2nd moment
    x -= eta * g / (np.sqrt(s) + eps) #divide by RMS magnitude
store("RMSprop", k, x)

# Adam ( momentum + RMSprop + bias correction)
eta, eps, beta1, beta2 = 0.15, 0.5, 0.8, 0.8
eta = 0.15 #learning rate
eps = 0.5 #stabiliser e
beta1 = 0.8 #momentum decay rate
beta2 = 0.8 #RMS decay rate
x = x0.copy()
m = np.zeros_like(x) #1st moment (mean grad)
v = np.zeros_like(x) #2nd moment (squared grad)
for k in range(1, max_iterations + 1):
    g = grad_f(x)
    if norm(g) < tol:
        break
    m = beta1 * m + (1 - beta1) * g #EWMA of gradient
    v = beta2 * v + (1 - beta2) * g**2 #EWMA of squared gradient
    m_hat = m / (1 - beta1**k) #bias-corrected 1st moment
    v_hat = v / (1 - beta2**k) #bias-corrected 2nd moment
    x -= eta * m_hat / (np.sqrt(v_hat) + eps) #adam update
store("Adam", k, x)

df = pd.DataFrame(results)
print(df.to_string(index=False))

                  Method  Iter    x0   x1     ‖x - x*‖  f(x_final)
             GD (a=0.02)  2436 -5.95 6.05 1.997032e-10      -18.05
Momentum (n=0.02, p=0.5)  1188 -5.95 6.05 1.992906e-10      -18.05
Nesterov (n=0.02, p=0.5)  5000   NaN  NaN          NaN         NaN
                 RMSprop  5000 -6.00 6.00 7.071068e-02      -18.00
                    Adam   216 -5.95 6.05 1.932902e-10      -18.05


<ipython-input-5-6a7615e84908>:16: RuntimeWarning: overflow encountered in matmul
  return A @ x + b
<ipython-input-5-6a7615e84908>:70: RuntimeWarning: invalid value encountered in add
  v = rho * v + g #same momentum update as before
